In [ ]:
#Imports utilizados
import sys
import keras as K
import numpy as np
import unicodedata
from sklearn.model_selection import train_test_split

from keras.layers import *
from keras import callbacks
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

In [ ]:
#Nome do arquivo a ser aberto e atualizado pra lowercase
filename = "junglebook"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [ ]:
#Cria um diretório para salvar os checkpoints e o log de treino
!mkdir "jb"
!mkdir "jk/log"

In [ ]:
raw_text = ''.join(ch for ch in unicodedata.normalize('NFKD', raw_text) 
    if not unicodedata.combining(ch))

In [ ]:
#Remove acentuação e pontuação de um texto
import re
raw_text = re.sub('[^a-zA-Z0-9 \n\.]', '', raw_text)

In [ ]:
#Exibe o texto após as modificações
raw_text

In [ ]:
#Separa todas as letras do dataset/text, ou seja, cria um dicionário enumerando cada letra presente no texto
#Para trabalharmos com essas letras precisamos transformá-las em números
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [ ]:
#Número de caracteres
n_chars = len(raw_text)
#Número de caracteres únicos 
n_vocab = len(char_to_int)

In [ ]:
print("Número de Caracteres: ",n_chars)
print("Número de Caracteres únicos: ",n_vocab)

In [ ]:
#Tamanho da Sequência que procuramos
#Nesse exemplo Utilizaremos uma sequência de 125, ou seja, usaremos 124 caracteres para prever o próximo
#Seguimos o padrão de one-hot-encoding que prediz cada letra individualmente entre as outras presentes.
seq_length = 125
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total de Padrões Apresentados: ", n_patterns)
print("Utlizando um tamanho de Sequência igual a :", seq_length)

In [ ]:
#Reshape da entrada pra treino
X = np.reshape(dataX, (n_patterns, seq_length, 1))
#Normalização
X = X / float(n_vocab)
y = np_utils.to_categorical(dataY)

In [ ]:
'''Usamos o train_test_split para separar 20% dos dados para teste, e assim
aplicar uma variedade melhor de métricas '''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
#Criação do Modelo
#Utilizamos uma rede razoavel de 2 camadas de 512 unidades de lstm para esse problema
#Dropout simples de 0.2 e uma Camada Densa que calcula a saída no total de letras no dataset
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(y.shape[1], activation='softmax'))

In [ ]:
#Callbacks utilizadas
filepath="jb/{loss:.3f}.hdf5"
#Checkpoint salva a cade época os pesos de nosso modelo caso ele tenha tido um resultado melhor que o anterior
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min', restore_best_weights=True)
#EarlyStopping comum, para o treino após 7 épocas sem melhoras na loss
stop = callbacks.EarlyStopping(monitor='loss', patience=7)
#CSVLogger, salva os dados/métricas de treino em um arquivo csv, usando um separador ","
log = callbacks.CSVLogger("jb/log/jb.csv", separator=",", append=True)
callbacks_list = [checkpoint,stop,log]

In [ ]:
#Define a perplexidade, uma métrica não integrada ainda ao Keras, porém é facilmente inserida
import keras.backend as K
def perplexity(y_true, y_pred):
    return K.exp(K.mean(K.categorical_crossentropy(y_true, y_pred)))

In [ ]:
#Compila nosso modelo, nesse exemplo utilizamos o optimzer adam e as metricas extras de acurácia e perplexidade
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy",perplexity])

In [ ]:
'''Inicia o treino de nosso modelo utiliando as ferramentes mostradas anteriormente, como:
-Callbacks
-Validation Data
Para esse treino utilizamos 300 épocas e batch_size de 50 para mais eficiência.
Dependendo do dataset, recomendo alterar esses valores para um treino mais rápido/simples ou o contrário =D
'''
history = model.fit(X_train, y_train, epochs=300, callbacks=callbacks_list, batch_size=50, validation_data=(X_test,y_test))

In [ ]:
#Carrega os pesos do melhor checkpoint do modelo
filename_w="jb/0.311.hdf5"
model.load_weights(filename_w)

In [ ]:
#Cria outra dicionário, inverso da operação anterior. Transforma os números em letras novamente
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
#Para gerarmos um texto, precisamos de um semente, e nisso escolhemos uma semente randomica de tamanho
#igual ao que escolhemos anteriormente, nesse caso 125
def pred_text(max_len):
  #Seleciona um texto randômico
  start = np.random.randint(0, len(dataX)-1) 
  pattern = dataX[start]
  print ("Semente randômica:")
  print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
  semente = "".join([int_to_char[value] for value in pattern])
  #Prediz os caracteres num range escolhido
  out = ""
  for i in range(max_len):
	  x = np.reshape(pattern, (1, len(pattern), 1))
	  x = x / float(n_vocab)
	  prediction = model.predict(x, verbose=0)
	  index = np.argmax(prediction)
	  result = int_to_char[index]
	  seq_in = [int_to_char[value] for value in pattern]
	  out+=result
	  pattern.append(index)
	  pattern = pattern[1:len(pattern)]
  return (semente + " -> " + out)

In [ ]:
#Chama a função pred_text para gerar um texto
saida_1 = pred_text(2000)
saida_2 = pred_text(1000)
saida_3 = pred_text(700)
saida_4 = pred_text(500)
saida_5 = pred_text(100)

In [ ]:
print("1 Resultado: ",saida_1)
print('------------------------') 
print("2 Resultado: " ,saida_2)
print('------------------------') 
print("3 Resultado: ",saida_3)
print('------------------------') 
print("4 Resultado: "  ,saida_4)
print('------------------------') 
print("5 Resultado: "  ,saida_4)
print('------------------------') 

In [ ]:
#Importa a biblioteca do Pandas
import pandas as pd

In [ ]:
#Lê o arquivo log que criamos utilizando a Callback CSVLogger
df = pd.read_csv("jb/log/jb.csv")

In [ ]:
#Transforma a primeira coluna no número de épocas, caso tenha tido problemas com Desconexão ou algo do tipo
for _ in range(len(df)):
  df["epoch"][_] = _+1 

In [ ]:
#Faz um plot e comparação da acurácia para o conjunto de treino e teste
df.plot(y=["accuracy","val_accuracy"],x='epoch', style=['g--','r--'], linewidth=2).set_facecolor('#ffff84')

In [ ]:
#Faz um plot e comparação da perplexidade para o conjunto de treino e teste
df.plot(y=["perplexity","val_perplexity"],x='epoch', style=['g--','r--'], linewidth=2).set_facecolor('#ffff84')

In [ ]:
#Faz um plot e comparação da perda para o conjunto de treino e teste
df.plot(y=["loss","val_loss"],x='epoch', style=['g--','r--'], linewidth=2).set_facecolor('#ffff84')